<a href="https://colab.research.google.com/github/victorviro/Machine-Learning-Python/blob/master/Monitoring_and_feedback_loop_MLOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of contents


1. [ℹ Introduction](#1)
2. [⌛ How often should models be retrained?](#2)
3. [📉 Understanding model degradation ](#3)
    1. [↩️ Ground truth evaluation ](#3.1)
    2. [➡️ Input drift detection ](#3.2)
5. [🔁 The feedback loop ](#5)
    1. [✍️ Logging](#5.1)
    2. [📏 Model evaluation store](#5.2)
    3. [🆚 Online evaluation](#5.3)
        1. [🥇🥈 Champion / Challenger](#5.3.1)
        2. [🅰🅱 A/B testing](#5.3.2)
6. [📓 References](#6)




## ℹ Introduction <a name="1"></a>

There are 🔢 **multiple reasons to 🚧 develop and 🚀 deploy a new ML model version**. Sometimes to refine business 🎯 objectives. Other times, the data scientists have come up with new features or a 👌 **better way to design the model**. Another common cause is model **performance 📉 degradation**.

When an ML model is 🚀 deployed in production, its performance can start ⬇ degrading over time and can impact 👎 negatively the business. That’s why model 🖥 **monitoring is a 🗝 crucial step** in the ML model 🔄 lifecycle. ML models need to be **monitored at ✌ two levels**:

| Level   |      ❓ Key questions      |
|----------|:-------------:|
| 💻 **Resource monitoring** |  - Is the **CPU, RAM, network usage, and disk space** as expected? Are requests being processed at the expected rate (**latency**)? |
| 📉 **Performance monitoring** | Is the model still an **accurate** representation of the pattern of new incoming data, and is it still performing as well as during its design phase? |


- <details>
  <summary>💻 <b>Resource monitoring</b></summary>
    
  It's is the traditional IT performance monitoring. The resource demands of ML models are not so different from traditional software. Overall, the existing expertise in **DevOps** teams **for monitoring and managing resources can be applied to ML models**.
 </details>

- <details>
  <summary>📉 <b>Performance monitoring</b></summary>
    
  It's inherent to ML. **How 👍 well a model performs** is a ↔ reflection of the data used to train it; particularly, **how representative the training data is of the live request data**. As the 🌍 world is constantly changing, a static model cannot 🥅 catch up with new patterns that are emerging without a constant source of new data. For example, the training data used to build a fraud detection model six months ago won’t reflect a new type of fraud that has started to occur in the last three months.
 </details>


<details>
  <summary>🕴️ <b>Business roles</b> also play on monitoring</summary>

  Some of their concerns might include ❓ questions like: 
  - Is the model delivering value to the enterprise?
  - Do the 👍 benefits of the model ⚖️ outweigh the 💰 cost of developing and 🚀 deploying the model? 
  The **KPIs** identified for the original business 🎯 objective **should be monitored 🤖 automatically** to prove value is ⬆ growing.
</details>

📉 Model performance **monitoring** tracks performance ⬇ degradation, and at an appropriate time, it can also **trigger the ↩️ retraining** of the model with more representative data. This 📄 notebook will delve into detail on how data teams should handle both 🖥 monitoring as well as subsequent ↩️ retraining.

## ⌛ How often should models be retrained? <a name="2"></a>

At some point, model **↩️ retraining is necessary**. How ⏳ soon models need to be retrained will depend on many ❓ factors:
- <details>
  <summary>The <b>domain</b>: how ✈ fast the real 🌍 world is changing? </summary>
  
  - In areas like real-time 💱 trading, models need to be 🔄 updated regularly to keep up with the changes inherent in these fields. 
  - On the other hand, physical models (like 🗣 voice recognition) are generally more stable, as the patterns don’t often abruptly change over time. 
</details>


- <details>
  <summary>Whether the 💰 <b>cost of ↩️ retraining is worth </b> the ⬆ improvement in performance</summary>
  
  For example, if it takes one week to run the whole data pipeline and ↩️ retrain the model, is it worth a 1% of ⬆ improvement in performance?
</details>


- <details>
  <summary>A <b>limited 🔢 number of training examples</b></summary>
  
  In these cases, the decision to retrain hinges on collecting enough new data
</details>

  <details>
  <summary>There are also two <b>organizational ↕️ bounds</b> to consider about 🔄 retraining frequency</summary>
  
  - An ↗️ *upper bound*: It's recommended retraining once every year to ensure that the team in charge has the skills to do it and to ensure that the computing ⚙ toolchain is still up.

  - A ↘ *lower bound*: Consider a model with ✈ quick feedback (e.g a recommendation engine). Some performance monitoring techniques (e.g shadow testing or A/B testing) that are used to 📉 monitor performance degradation are statistical techniques, so it takes some ⌛ time to gather the required information (at minimum, one day). This necessarily sets a lower bound to the retraining period. The **lag between prediction time and ground truth obtention time** is 🗝 key to define this ↘ lower bound. It's ⚠ risky to use a model when it's likely that it drifts faster than this lag. If the model retraining occurs more often than the lag, there will be almost no impact on the performance of the model. 
</details>


It's ⚡️ critical to know the drift and **accuracy of 🚀 deployed models** by setting up ⚙ **processes** for 🖥 **monitoring and 📧 notifications**. An ideal scenario would be a pipeline that 🤖 automatically triggers ✅ checks for the 📉 degradation of model performance. 📧 Notifications are not necessarily to kick off an automated process of ↩️ retraining, ✅ validation, and 🚀 deployment. **Model performance can change due to different reasons, and ↩️ retraining may not always be the answer**. The point is to 🔔 alert the data scientist, who can then 🔎 diagnose the issue and 🤔 evaluate the situation. 

Practically, every 🚀 deployed model should come with monitoring 📏 metrics and corresponding warning 🔢 thresholds to detect meaningful business performance ⬇ drops as ✈ quickly as possible.

## 📉 Understanding model degradation <a name="3"></a>



**How** can we ❗ **notice a model’s performance is ⬇ degrading?** There are ✌ **two common approaches**: ground truth evaluation and input drift detection.

### ↩️ Ground truth evaluation <a name="3.1"></a>

The **ground truth is the correct answer** to the ❓ question the model was asked to solve (e.g. is this credit card transaction actually fraudulent?). 

<details>
  <summary>The ⌛ time that passes between prediction time and ground truth obtention depends on the use case</summary>
  
  - **Sometimes** ground truth **is obtained 🚄 rapidly after a prediction** (e.g. in advertisement recommendation systems, users likely 🖱 click on the ads within seconds). 

  - However, **in many use cases, obtaining the ground truth is much 🐌 slower**. For example, a model that predicted that a transaction was not 🚫 fraudulent, how can this be ✅ confirmed? It'll be reported probably by the cardholder when he reviews their monthly transactions, but this could happen up to a month after the event. In this case, ground truth won't enable monitoring performance accurately on a daily basis. **If the situation requires 🏎 rapid feedback, then input drift detection may be a better approach**. 
</details>



With the new ground truth ↩️ collected, the ➡ next step is to **compute the performance of the model based on ground truth and compare** it with the 📏 metrics. When the difference surpasses a 🎚 threshold, the model can be deemed as outdated, and it should then be 🔄 retrained.

The 📐 metrics to be 📊 monitored can be of ✌ two varieties:

| Type of metric   |      Examples      |  Adventages | Drawbacks |
|----------|:-------------:|------:|-----:|
| 🔢 **Statistical metrics**|  Accuracy, ROC AUC... | Domain agnostic. Easy for data scientists to set 🎚 thresholds | Drop may be statistically significant without having any noticeable impact |
| 💼 **Business metrics** | Cost/benefit assessment | They have a **monetary value** | |


> **When available, ground truth monitoring is the ⭐ best solution**. 

Ground truth monitoring involves three main ⚡️ **challenges**:

- <details>
  <summary>↩️ Ground truth is not always immediately available</summary>
  
  It can mean significant 💲 economic ⬇ loss if the model is degrading 🚄 quickly.
</details>


- <details>
  <summary>🔀 Scattered information</summary>
  
  **To compute the 📏 performance** of the 🚀 deployed model on new data, we need 🔗 **match ground truth with the corresponding observation**. In many production environments, this is hard because these two pieces of **information** are generated and stored **in different systems** and at different timestamps.
</details>


- <details>
  <summary>🌓 Partially available ground truth</summary>
  
  In some situations, it's 💵 **expensive to retrieve the ground truth for all the observations**, which means 👉 choosing which **samples** to 🏷 label and this can inadvertently **introduce bias** into the system. Fraud detection is an example. As transactions need to be examined manually, it seems reasonable to establish ground truth for only suspect cases (cases where the model gives a ⬆ high probability of fraud). However, this introduces bias since fraud patterns that were never captured by the model (those that the model predicts a ⬇ low fraud probability) will be ignored. The 🏷 labeled sample subset must **cover all the possible future predictions** so that the model continues to generalize 👍 well.
</details>

### ➡️ Input drift detection <a name="3.2"></a>

> A model will predict 👍 accurately if the data it was trained on is an accurate ↔️ reflection of the real 🌍 world.

So if a **comparison** of recent production data 🆚 against the training data **shows differences, then it's likely that the model performance is compromised**. The 🤩 beauty of this approach is that all the data required already exists (**no need to 🤚 wait for ground truth**).

But, **how can drift be detected?** There are ✌ two common approaches. 

- <details>
  <summary>🔢 Univariate <b>statistical tests</b></summary>
  
  For each feature, a statistical ✅ test is applied on data from the training data distribution and the production data distribution. A ⚠️ warning will be raised when the results of those tests are significant. The most commonly used tests are:
   - For 🔢 **continuous features, the [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test)** is a non-parametric hypothesis test that ✅ checks whether **two samples come from the same distribution**. 

  - For 🏷 **categorical features, the [Chi-squared test](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)** checks whether the observed frequencies for a categorical feature in the target data match the expected frequencies seen from the source data.

  **P-values** help to detect drift but they 👎 **do not quantify the 🎚 level of the effect** (they can detect small changes which may be impactless). So it's necessary to complement them with 💼 business-significant 📏 metrics. For example, on a large dataset, the average age may have significantly drifted from a statistical perspective, but if the drift is only a few months, this is probably insignificant for many business cases.
</details>


- <details>
  <summary><b>Domain 🏷 Classifier</b></summary>
  
  We **train a model that tries to discriminate between the training dataset** **and** the **production dataset**. That is, a **classifier that aims at predicting data’s origin**. The performance of the model can then be considered as a 📏 metric for the drift 🎚 level.

 **If this model is 👍 successful in its task**, and thus has a ⬆ high drift score, it implies that **data used at training time and new data can be distinguished**, so it’s fair to say that the **new data has drifted**. To 🔎 identify the features that are responsible for the drift, one can use the 📊 **feature importance** of the trained model.
</details>

  <details>
  <summary>Statistical tests 🆚 domain 🏷 classifier</b></summary>
  
  The 👉 choice between these approaches depends on the required 🎚 level of interpretability. 
  - If **explainable** methods are **required**, 🔢 **univariate statistical tests** are prefered. 
  - If **complex drift** involving several features simultaneously is expected, the 🏷 **domain classifier** approach may be a 👍 good option.
</details>




Data drift can be identified for different features:

- <details>
  <summary><b>Drift attributed to the 🎯 target feature</b></summary>
  
  It's important to 🔎 identify because it often directly impacts the 💼 business (e.g. in a credit score system, if the scores are ⬇ lower overall, the number of awarded loans will likely be lower, and therefore the 💰 revenues). 
</details>


- <details>
  <summary><b>Drift attributed to features</b></summary>
  
  It's useful to mitigate the impact, as it may hint at the need for:

  - ⚖️ **Reweighting** according to this feature (e.g., if customers above 60 now represent 60% of users but it was only 30% in the train set, then we can ✖️ double their weight and ↩️ retrain the model).
</details>



In all cases, it is **unlikely** that 🤖 **automatic actions** exist if drift is detected. It could happen if it is not 💰 costly to 🚀 deploy retrained models.

## 🔁 The feedback loop <a name="5"></a>

The data 🔁 feedback loop ([Continuous Delivery for ML](https://martinfowler.com/articles/cd4ml.html)), shows the idea that **information from the production environment ↩️ flows back** to the model prototyping environment **for further ⬆ improvement**. 


<details>
  <summary>Image</summary>

  ![](https://i.ibb.co/SNnW3vz/0701.png)
  - Data collected in the 🖥 Monitoring and 🔎 Observability phase is sent ↩️ to the 🚧 Model Building phase. 
  - From there, the system analyzes whether the model is 👍 working as expected. 
    - If it's the case, no 🎬 action is required. 
    - If the model’s performance is ⬇ degrading, an update will be triggered, either 🤖 automatically or 🤚 manually by the data scientist. This usually means either ↩️ retraining the model with new labeled data or developing a new model with ➕ additional features.
</details>



The 🎯 goal is 🥅 capture the emerging patterns and **make sure that the business is not 👎 negatively impacted**. In ➕ addition to the concepts previously discussed, three components that we discuss next are 🗝 critical:



| Component |      Description      |
|----------|:-------------:|
| ✍️ **Logging system** |  It collects data from several production servers |
| 📏 **Model evaluation store**  | It performs **versioning** and 📏 evaluation between different model versions | 
| 🆚 **Online evaluation**  | It performs **model 🆚 comparison** on production environments, either with shadow or A/B testing | 

### ✍️ Logging <a name="5.1"></a>



🖥 **Monitoring a live system means 📝 collecting and aggregating data about its states**. Data from these environments need to be centralized in a place to be 📈 analyzed and monitored (either 🤖 automatically or 🤚 manually). This enables continuous ⬆ improvement of the ML system. 

An **event log** of an ML system is a **record with a timestamp and the following information**:


| ℹ Information | 🤔 Description |
|----------|:-------------:|
| *Model metadata* | Identification of the model and the version |
| *Model inputs*  | Feature values of new observations | 
| *Model outputs* | Predictions made by the model | 
| *System action* | The 🎬 action made by the system based on the prediction (e.g. in fraud detection, when the model gives a ⬆ high probability, the system can 🚫 block the transaction) | 
| *Model explanation* | Explanation for the prediction (required in highly regulated domains such as 🏦 finance) | 

Nowadays, as production 🏗 infrastructures are getting more complex with several models 🚀 deployed simultaneously across several servers, an effective ✍️ logging system is more important than ever, but there are some requirements and challenges.


<details>
  <summary>📜 Requirements</summary>

  - The system can **access and ↩️ retrieve scoring logs from multiple servers**, either in real-time inference or in batch inference.

  - When a **model is 🚀 deployed on multiple servers**, the system can handle the 🔗 **mapping and aggregation of all information** per model across servers.
</details>


<details>
  <summary>⚡️ Challenges</summary>

  - For **large-scale ML applications**, the number of raw event logs generated can be an issue. However, as the 🎯 goal of monitoring is usually to estimate aggregate 📏 metrics, **saving only a subset of the predictions** may be acceptable in many cases. 
</details>





### 📏 Model evaluation store <a name="5.2"></a>



With a ✍️ logging system in place, it periodically fetches data from the production environment for 📊 monitoring. One day, the data drift 🔔 alert is triggered. The model performance is ⬇ degrading. After 🔎 review, data scientists decide to ↩️ retrain the model. With several trained candidate models, the ⏭ next step is to 🆚 **compare** them **with the 🚀 deployed model**. 

In practice, this means 📏 evaluating all the models (the candidates as well as the 🚀 deployed model) on the same dataset. **If one of the candidate models 🥇 outperforms the active model**, there are two ways to proceed: **update the model** on the production environment or move to an **online evaluation**.

The model evaluation store is a structure that centralizes the data related to the model lifecycle to allow 🆚 **comparing** multiple, newly trained **model versions against versions of 🚀 deployed models** or other models on 🏷 labeled data. It also allows to ✍ track model performance over time.



### 🆚 Online evaluation <a name="5.3"></a>

There are ✌ two main modes of online evaluation:

- 🥇🥈 **Champion/challenger** (or shadow testing), where the 🥈 candidate model shadows the 🥇 deployed model and **scores the same live requests**.

- 🅰/🅱 **testing**, where the 🥈 candidate model **scores a portion of the live requests** and the 🥇 deployed model scores the others.

Note that both cases **require ↩️ ground truth**, so the evaluation will necessarily take ⌛ longer than the lag between prediction and ground truth obtention. In addition, whenever 🥇🥈 **shadow testing** is possible, it should be used **over A/B testing** because it is far simpler to understand and to set up, and what’s more, it 🔎 **detects differences more ✈ quickly**.  




#### 🥇🥈 Champion/Challenger (shadow testing) <a name="5.3.1"></a>



🥇🥈 Champion/Challenger  involves 🚀 **deploying additional models** (the 🥈 **challengers**) to the production environment. These models receive and **score the same incoming requests as the active one** (the 🥇 **champion model**). However, they **don't ↩️ return any response** or prediction to the system: that’s still the job of the 🥇 champion model (the predictions are **simply ✍ logged** for further 📊 analysis). This setup allows ✌️ two things:

- <details>
  <summary>✔️ Verification that the performance of the new models are 👍 better</summary>

  As the two models are scoring on the same data, there is a **direct 🆚 comparison of the metric of the two models in the production environment**. This **could also be done 📴 offline** by using the new models on the dataset made of new requests scored by the 🥇 champion model.
  </details>

- <details>
  <summary>📐 Measurement of how the model handles the ⚙ realistic load</summary>

  As the new model can have new features or preprocessing techniques, the ⏳ **prediction time for a request won’t be the same as that of the original one**. This is the main 👍 **advantage of doing it online**.
  </details>


📜 Requisites

- <details>
  <summary>✍ Same information must be logged</summary>

  To be able to 🆚 compare both champion/challenger models, the **same information must be ✍ logged for both** (input and output data, ⏳ processing time...).
</details>

- <details>
  <summary>⏳ Time deployed</summary>

  - To make a clear decision about what model performs 👍 better, both models should be deployed ⏳ long so that enough predictions are made and metric fluctuations due to randomness are dampened (the difference is significant). This can be assessed 📉 graphically by checking that the **metric estimations** are not giggling anymore or by doing a ✅ **statistical test** (a paired sample T-test).
</details>


🤔 Considerations
- <details>
  <summary>🖥 Server performance</summary>

  If two **memory-intensive models** are called synchronously, they **can 🐌 slow the system down**, impacting 👎 negatively the user experience but also ❌ **corrupt the data collected** about the functioning of the models.
</details>

- <details>
  <summary>🗣 Communication with external system</summary>

  If the two models use an **external API to enrich** their features, that 2️⃣ **doubles the number of requests** to these services, thus doubling 💲 costs. If that API service has some **caching system** in place, then the **second request** will be processed much 🏎 **faster** than the first one, which can **bias** the result when 🆚 comparing the total ⌛ **prediction time** of the two models. 
  
  **Note**: the 🥈 **challenger may be used only for a random subset of the incoming requests**, which will alleviate the ⚙ load at the expense of ⬆ increased ⌛ time before a conclusion can be drawn.
</details>

- <details>
  <summary>🎬 Challenger model does not influence the system's actions anyway</summary>

  - **When the challenger model** encounters an unexpected 🐛 issue and  ❌ **fails, the production environment won't experience any discontinuation or degradation** in terms of ⏳ response time.

 - 🎬 **Actions** taken by the system **depend only on the prediction of the 🥇 champion model**, and they happen only once. For example, in a fraud detection use case, imagine that by mistake the challenger model is plugged directly into the system, charging each transaction twice (a 💥 catastrophic scenario).
</details>

#### 🅰🅱 A/B testing <a name="5.3.2"></a>

Unlike shadow testing, **with A/B testing, the 🥈 candidate model 🔙 returns predictions for certain requests, and the original model handles the other ones**. Once the test ⏲ period is over, ✅ **statistical tests 🆚 compare the performance of the two models**, and teams can make a decision. 

For ML models, A/B testing **should be used only when 🥇🥈 champion/challenger is not possible**. This might happen when:

- <details>
  <summary>↩️ The ground truth cannot be evaluated for both models</summary>

  For example:
  - For a recommendation engine, the prediction ➡ gives a list of items on which a given customer is likely to 🖱 click if they are presented. Therefore, it is impossible to know if the customer would have clicked if it is not presented, so some kind of A/B testing will have to be done. 
  - For a fraud detection model, as 🏋 heavy work is needed to ↩️ obtain the ground truth, it may not be possible to do it for the positive predictions of two models because it would ⬆ increase the workload too much, as some frauds are detected by only one model. As a result, randomly applying only the B model to a small fraction of the requests will allow for the workload to remain ➖ constant.
</details>


- <details>
  <summary> 🎯 The objective to optimize is only indirectly related to the performance of the prediction</summary>

  Imagine an ad engine based on an ML model that predicts if a user will 🖱 click on the ad. Now imagine that it is evaluated on the buy rate, i.e., if the user bought the product. It is not possible to record the reaction of the 🧑 user for two different models, so in this case, A/B testing is the only way.
</details>


Steps:

- ⬅️ Before the A/B test: 
  - <details>
    <summary> 🎯 Define a clear goal</summary>

    A 📏 quantitative **business metric** that needs to be optimized. For example, 🖱 click-through rate. 
  </details>

  - <details>
    <summary>👬 Define a precise population</summary>

    Carefully choosing a segment for the ✅ test along with a ✂ splitting strategy that assures **no bias between groups** (experimental design). This may be a random split, but it may be more complex. For example, the situation might dictate that all the requests of a particular customer are handled by the same model.
  </details>

  - <details>
    <summary>📜 Define the statistical protocol</summary>

    The resulting metrics are 🆚 compared using ✅ statistical tests. To make the conclusion robust, teams need to define beforehand the 📏 **sample size** for the desired minimum effect size, which is the minimum difference between the two model's performance metrics. Teams must also fix a ⌛ **test duration**. With similar sample sizes, the power to detect meaningful differences will be lower than with champion/challenger because **unpaired sample tests have to be used**.
  </details>


- During the A/B test: 

  - <details>
    <summary>⌛ Test period</summary>

    - Fixed-horizon test (frequentist): If the **experiment is 🛑 stopped before the test duration is over** (even if the ✅ statistical test starts to return a significant metric difference), it can produce **biased results** due to cherry-picking the desired outcome (**p-hacking**). By other hand, the test is running live, and in a commercial environment, every bad prediction is likely to cost 💲 money, so not being able to 🤚 stop a test early could be expensive.

    - **Multi-arm bandit** tests (bayesian) draws conclusions 🏎 quicker. It's **adaptive** (the algorithm that decides the split between models adapts according to live results and ⬇ **reduces the ⚙ workload of underperforming models**). While this testing is **more complex**, it can reduce the business 💲 cost of sending 🚦 traffic to a 👎 poorly performing model.
  </details>


- <details>
  <summary>➡️ After the A/B test: </summary>

  Once the ⌛ test duration is over, 🔍 check the collected data to make sure that the quality is 👍 good. From there, run the ✅ statistical tests; if the 📏 metric difference is statistically significant in favor of the 🥈 candidate model, the original model can be ⏩ replaced with the new version.
</details>

# 📓 References <a name="6"></a>

- [MLOps: Continuous delivery and automation pipelines in machine learning](https://cloud.google.com/solutions/machine-learning/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)

- [Introducing MLOps](https://www.oreilly.com/library/view/introducing-mlops/9781492083283/)

- [MLOps.Community](https://www.youtube.com/channel/UCG6qpjVnBTTT8wLGBygANOQ)